<a href="https://colab.research.google.com/github/tonytani37/SpicySessions/blob/main/csv_to_json_convert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
# prompt: /content/spicy_sessions_songs.jsonをcsvファイルにする それぞれのレコードにはセッション情報という配列が含まれるが、それらはセッション曲ごとに１レコードにする　なおセッション情報がない場合には空の配列とする

import json
import pandas as pd

def json_to_csv(json_file_path, csv_file_path):
    try:
        with open(json_file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
    except FileNotFoundError:
        print(f"Error: File not found at {json_file_path}")
        return
    except json.JSONDecodeError:
        print(f"Error: Invalid JSON format in {json_file_path}")
        return

    rows = []
    for record in data:
        sessions = record.get('セッション情報', [])  # セッション情報が存在しない場合の処理を追加
        if not sessions:
          sessions = []
        for session in sessions:
            new_row = {**record, **session}
            # セッション情報以外のキーを新しい行に追加
            del new_row['セッション情報']
            rows.append(new_row)

    df = pd.DataFrame(rows)
    df.to_csv(csv_file_path, index=False, encoding='utf-8-sig') # BOM付きで保存
    print(f"Successfully converted {json_file_path} to {csv_file_path}")

json_to_csv('/content/spicy_sessions_songs.json', '/content/spicy_sessions_songs.csv')


Successfully converted /content/spicy_sessions_songs.json to /content/spicy_sessions_songs.csv


In [16]:
# prompt: /content/spicy_sessions_songs.csvを読み込んで、["回", "放送日", "放送時間", "放送タイトル", "カレー", "参考"]でグルーピングする。それをグルーピングしたキーを基本に同じ回でセッション情報をまとめる　その時備考は無視する

import json
import pandas as pd

# CSVファイルの読み込み
csv_path = "/content/spicy_sessions_songs.csv"  # ←ファイルパスを適宜変更してください
df = pd.read_csv(csv_path)

df["参考"].fillna("　", inplace=True) #未入力（NaNがある場合は全角スペースとする）

# セッション情報にまとめる列
session_cols = ["セッション曲", "オリジナル", "歌唱", "演奏者"]

# 回などの情報でグループ化
grouped = df.groupby(["回", "放送日", "放送時間", "放送タイトル", "カレー", "参考"])

# JSONデータ構築
json_data = []
for keys, group in grouped:
    session_info = group[session_cols].to_dict(orient="records")
    entry = {
        "回": int(keys[0]),  # int型に変換
        "放送日": keys[1],
        "放送時間": keys[2],
        "放送タイトル": keys[3],
        "カレー": keys[4],
        "参考": keys[5],
        "セッション情報": session_info
    }
    json_data.append(entry)

# JSONファイルに保存
output_path = "/content/spicy_sessions_songs.json"  # 出力ファイル名
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(json_data, f, ensure_ascii=False, indent=2)

print(f"変換完了！JSONファイル保存先: {output_path}")


変換完了！JSONファイル保存先: /content/spicy_sessions_songs.json


<ipython-input-16-0355413cfdd4>:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["参考"].fillna("　", inplace=True)
